# Prepare the instance files for model building
**Conda env:** /home/seguraab/miniconda3/envs/py310

## Using 20250403_melissa_ara_data
For classification models:
1. Binary label for each trait: neg & pos GI as 1, no GI as 0 [sample from bins for the no GI]
2. Binary label (all traits included): 1 if GI in at least one trait; 0 if no GI in any trait


[Don't do for now] 3. Multiclass label for each trait: neg GI, pos GI, no GI [sample from bins]


[Don't do for now] 4. Multiclass label (all traits included): all neg GI for any trait, all pos GI for any trait, no GI in any trait. [What to do if a GI is neg, but pos for a different trait? Perhaps only combine the \<label\>_\<transformations\> columns per label]

[Don't do for now] Brianna suggested instead of multiclass, to create binary labels (0/1 for neg GI; 0/1 for pos GI; 0/1 for not-detected), these could be used in a multi-output classification. Instead of multiclass classification.

5. Relax the p-value to 0.1 and re-create the above 4 labels

For regression models:
1. 

I'll combine all of these labels into one file.

In [1]:
# Read in epistasis data
import os
import pandas as pd
dir_path = '/home/seguraab/ara-kinase-prediction/data/20250403_melissa_ara_data/corrected_data'
data_linear = pd.read_csv(f'{dir_path}/fitness_data_04032025_epistasis_linear_with_no_int.csv') # from linear model
data_mani = [f for f in os.listdir(dir_path) if f.endswith('emmeans_epistasis.tsv')] # Mani 2008 definitions

In [2]:
import numpy as np
def sample_neg_set_bins(neg_set, num_bins=10, num_samples=50):
    '''Sample equally from the negative set distribution using bins'''
    neg_set = neg_set.copy(deep=True)
    
    # Create bins
    neg_set_bins = np.linspace(neg_set.e_est.min(), neg_set.e_est.max(), num_bins + 1)
    neg_set['bins'] = pd.cut(neg_set.e_est, bins=neg_set_bins, include_lowest=True)
    
    # Sample equally from each bin
    num_samp_per_bin = num_samples // num_bins
    sampled_data = neg_set.groupby('bins').apply(
        lambda x: x.sample(n=min(len(x), num_samp_per_bin), random_state=2305))
    sampled_data = sampled_data.reset_index(drop=True)
    
    return sampled_data


def make_imbalanced_binary_labels_from_linear_model(data_linear):
    '''Generate imbalanced binary labels using epistasis estimates from the
    linear model'''
    
    # Store labels in a dictionary
    labels = {}
    
    for cutoff, pval in zip([0.05, 0.1], ['p05', 'p1']):
        # Subset the data by p-value cutoffs
        neg_set = data_linear[data_linear.pval_e >= cutoff]
        pos_set = data_linear[data_linear.pval_e < cutoff]
        # pos_set_size = pos_set.groupby('Label').count() # size of positive set, to balance negative set
        
        # Binary label for each trait------------------------------------------#
        for label in pos_set['Label'].unique():
            # positive set for this label
            label_pos_set = pos_set[pos_set['Label'] == label].copy(deep=True)
            label_pos_set['ML_label'] = 1
            
            '''Don't need this since the RF & XGB code does this already:
            # balanced negative set label
            balanced_neg_set = sample_neg_set_bins(neg_set[neg_set['Label'] == label],
                num_samples=pos_set_size.loc[label, 'Set'])
            balanced_neg_set['ML_label'] = 0
            '''
            
            # negative set for this label
            label_neg_set = neg_set[neg_set['Label'] == label].copy(deep=True)
            label_neg_set['ML_label'] = 0
            
            # ensure set numbers don't overlap
            assert label_pos_set.Set.isin(label_neg_set.Set).sum() == 0, \
                f'Overlapping sets for {label} in {pval}'
            
            # combine the positive and negative sets and assign name to the label
            labels[f'binary_{label}_{pval}'] = pd.concat([
                label_neg_set.set_index('Set')['ML_label'],
                label_pos_set.set_index('Set')['ML_label']], axis=0)
            del label_pos_set, label_neg_set
        
        # Imbalanced binary label for all traits combined---------------------------------#
        # note: not all sets fell under the p-value cutoff for all the traits,
        # but since the genes are interacting, then I am setting the label to 1.
        # The code below could be simpler, but I wanted to make it this point clear.
        pos_set['ML_label'] = 1
        pos_set = pos_set.pivot(index='Set', columns='Label', values='ML_label').any(axis=1) # will have NaN because of the note above
        
        # set all positive set instances to 1 if they are not all 1
        if pos_set.value_counts().get(True)!=len(pos_set):
            pos_set = pd.Series(1, index=pos_set[pos_set].index)
        
        # negative set
        neg_set = data_linear[~data_linear.Set.isin(pos_set.index)].copy(deep=True)
        neg_set['ML_label'] = 0
        neg_set = neg_set.pivot(index='Set', columns='Label', values='ML_label').any(axis=1).astype(int)
        
        # combine the positive and negative sets and assign name to the label
        labels[f'binary_combined_{pval}'] = pd.concat([neg_set, pos_set], axis=0)
    
    return labels

def make_imbalanced_multiclass_labels_from_linear_model(data_linear):
    # Store labels in a dictionary
    labels = {}
    
    for cutoff, pval in zip([0.05, 0.1], ['p05', 'p1']):
        # Subset the data by p-value cutoffs
        neg_set = data_linear[data_linear.pval_e >= cutoff] # not_detected
        pos_set = data_linear[data_linear.pval_e < cutoff] # positive & negative GIs
        
        # Binary label for each trait------------------------------------------#
        for label in pos_set['Label'].unique():
            # significant positive and negative GIs classes for this label
            label_pos_set = pos_set[pos_set['Label'] == label].copy(deep=True)
            label_pos_set['ML_label'] = label_pos_set['Epistasis_Direction'].str.lower()
            
            assert label_pos_set['ML_label'].nunique() == 2, \
                f'Label {label} has more than 2 unique values in Epistasis_Direction for the positive set'
            
            # insignificant gene pairs (no GI) class for this label
            label_neg_set = neg_set[neg_set['Label'] == label].copy(deep=True)
            label_neg_set['ML_label'] = label_neg_set['Epistasis_Direction'].str.lower()
            label_neg_set['ML_label'] = label_neg_set['ML_label'].str.replace(' ', '_')
            
            assert label_neg_set['ML_label'].nunique() == 1, \
                f'Label {label} has more than 1 unique value in Epistasis_Direction for the negative set'
                
            # ensure set numbers don't overlap
            assert label_pos_set.Set.isin(label_neg_set.Set).sum() == 0, f'Overlapping sets for {label} in {pval}'
            
            # combine the positive and negative sets and assign name to the label
            labels[f'multiclass_{label}_{pval}'] = pd.concat([
                label_neg_set.set_index('Set')['ML_label'],
                label_pos_set.set_index('Set')['ML_label']], axis=0)
            del label_pos_set, label_neg_set
        
        # Imbalanced binary label for all traits combined---------------------------------#
        # note: not all sets fell under the p-value cutoff for all the traits,
        # but since the genes are interacting, then I am setting the label to 1.
        # The code below could be simpler, but I wanted to make it this point clear.
        pos_set = pos_set.pivot(index='Set', columns='Label', values='Epistasis_Direction') # will have NaN because of the note above
    
        # set all positive set instances to 1 if they are not all 1
        if pos_set.value_counts().get(True)!=len(pos_set):
            pos_set = pd.Series(1, index=pos_set[pos_set].index)
        
        # negative set
        neg_set = data_linear[~data_linear.Set.isin(pos_set.index)].copy(deep=True)
        neg_set['ML_label'] = 0
        neg_set = neg_set.pivot(index='Set', columns='Label', values='ML_label').any(axis=1).astype(int)
        
        # combine the positive and negative sets and assign name to the label
        labels[f'binary_combined_{pval}'] = pd.concat([neg_set, pos_set], axis=0)
    
    return labels

In [ ]:
# Note: there are sets with NaN values, that is because they were missing genotypes,
# thus a linear model could not be fit. These sets will need to be removed within the
# model training code.
binary_labels = pd.DataFrame(make_imbalanced_binary_labels_from_linear_model(data_linear))

# Insert gene names
gene_names = pd.read_excel('/home/seguraab/ara-kinase-prediction/data/20240917_melissa_ara_data/fitness_data_for_Kenia_09172024.xlsx', sheet_name='gene_names')
gene_names['Line'] = gene_names.Line.astype(str)
gene_names = gene_names[['Line', 'MA', 'MB']].set_index('Line').to_dict()
binary_labels.insert(0, 'gene1', binary_labels.index.map(gene_names['MA']).str.upper().str.strip())
binary_labels.insert(1, 'gene2', binary_labels.index.map(gene_names['MB']).str.upper().str.strip())
# binary_labels.loc[~binary_labels.index.isin(gene_names['MA'].keys())] # Will drop sets 77, 793, 813, 825, and 86
binary_labels.dropna(subset=['gene1', 'gene2'], inplace=True)

# Save to file
binary_labels.to_csv(f'{dir_path}/binary_labels_from_linear_model.csv', index=True)


/tmp/ipykernel_757464/1136230926.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_set['ML_label'] = 1
/tmp/ipykernel_757464/1136230926.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_set['ML_label'] = 1


## Using 20240725_melissa_ara_data/interactions_fitness.txt
For classification models:  
1. Train/Val/Test: Melissa's alpha whole genome duplication gene pairs as instances. The label is 0 for no interaction and 1 for interaction.  
2. Predict: Cusack 2021 kinase family gene pairs
3. Predict: Kinase gene pairs from Araport11 (Won't do this)
4. Predict: Kinase gene pairs from TAIR10

For regression models:  
1. Train/Val/Test: Melissa's gene pairs with the corrected total seed count (TSC) as the label
2. Predict: All possible kinase family gene pairs
3. Predict: Kinase gene pairs from Araport11 (Won't do this)
4. Predict: Kinase gene pairs from TAIR10

Additional instance files:  
- All possible gene pair combinations as instances from the A. thaliana TAIR10 genome  
- Single genes as instances from TAIR10  

### Prepare instance file no. 1

In [1]:
import pandas as pd

# Read in the gene pairs from the two datasets
ara_m = pd.read_csv('../data/20240725_melissa_ara_data/interactions_fitness.txt', sep='\t') # Melissa's gene pairs
ara_m.rename(columns={'MA': 'gene1', 'MB': 'gene2'}, inplace=True)
kinases = pd.read_csv('../data/2021_cusack_data/Dataset_4.txt', sep='\t')
kinases = kinases.loc[kinases.Class=='test'] # these are the kinase family gene pairs

# Split the gene pair identifier into two columns
instances = kinases.pair_ID.str.split('_', expand=True)
instances.columns = ['gene1', 'gene2']

# Merge the instances
instances = pd.concat([instances, ara_m[['gene1', 'gene2']]], axis=0, ignore_index=True)
instances['gene1'] = instances['gene1'].str.upper()
instances['gene2'] = instances['gene2'].str.upper()
print(instances.shape) # (10265, 2)

# Check for duplicate gene pairs
instances = instances.apply(lambda x: sorted(x), axis=1) # sort the gene pairs
instances = pd.DataFrame(instances.to_list(), columns=['gene1', 'gene2'])
instances.drop_duplicates(inplace=True)
print(instances.shape) # (10250, 2); 15 gene pairs overlapped with ara_m

# Save the instances
# instances.to_csv('../data/instances_dataset_1.txt', sep='\t', index=False)
instances

(10265, 2)
(10250, 2)


,gene1,gene2
0,AT3G46420,AT4G20450
1,AT5G01820,AT5G57630
2,AT2G37050,AT5G59660
3,AT3G17840,AT3G51740
4,AT1G11410,AT4G23190
...,...,...
10259,AT1G23380,AT1G70510
10260,AT1G26790,AT1G69570
10261,AT1G16060,AT1G79700
10262,AT1G21410,AT1G77000


In [3]:
import os

os.chdir('/home/seguraab/ara-kinase-prediction')
kinases = pd.read_csv('data/2021_cusack_data/Dataset_4.txt', sep='\t')

# sort instance identifiers
sorted_IDs = kinases['pair_ID'].str.split('_').apply(sorted).str.join('_')
sum(sorted_IDs == kinases.pair_ID.values) # they're exactly the same, good!

# Note: I am going to generate features for Dataset_4.txt (saved to ara-kinase-prediction/data/2021_cusack_data/Dataset_4_Features/)

10300

##### Create test set files
These test sets will be used to evaluate model performance in a "round robin" style.
For the cross-validation during model training, I will use:
1. Leave-One-Out cross-validation with the remaining 9 folds
2. Normal 9-fold cross-validation, excluding the test set for each round robin iteration

In [ ]:
# Drop the duplicate instance from ara_m before assigning instances to CV folds
ara_m = pd.read_csv('../data/20240725_melissa_ara_data/interactions_fitness.txt', sep='\t') # Melissa's gene pairs
ara_m.index = np.sort(ara_m[['MA', 'MB']], axis=1) # sort the gene pairs
ara_m.index = ara_m.index.map(tuple) # convert to tuples
ara_m.index[ara_m.index.duplicated()] # At1g18620 At1g74160 is duplicated (set 703)
ara_m_no_dups = ara_m.loc[ara_m.index.drop_duplicates()] 


# Convert instances to uppercase
ara_m_no_dups.index = ara_m_no_dups.index.set_levels(ara_m_no_dups.index.levels[0].map(str.upper), level=0)
ara_m_no_dups.index = ara_m_no_dups.index.set_levels(ara_m_no_dups.index.levels[1].map(str.upper), level=1)
ara_m_no_dups = ara_m_no_dups.loc[ara_m_no_dups.Set != 703]

# Assign instances to CV folds
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=20240909)
for i, (train_idx, test_idx) in enumerate(folds.split(ara_m_no_dups, ara_m_no_dups['Interaction'])):
	fold_i = ara_m_no_dups.loc[ara_m_no_dups.index[test_idx]].index.to_frame(index=False)
	
	with open(f'../data/test_sets_clf/test_ara_m_fold_{i}.txt', 'w') as out:
		for j in range(fold_i.shape[0]):
			out.write(f'{fold_i.iloc[j,0]}_{fold_i.iloc[j,1]}\n')


### Prepare instance file of TAIR10 Kinase genes

In [ ]:
import itertools
import datatable as dt
import numpy as np

# Kinase genes from NCBI
genes = pd.read_csv("../data/Kinase_genes/kinase-athaliana-ncbi.tsv", sep="\t")
k_id = genes.apply(lambda x: x["Aliases"].split(",")[0].strip() if isinstance(x["Aliases"], str) else x, axis=1)
print(k_id.apply(type).unique()) # type: ignore
genes.iloc[k_id.loc[k_id.apply(lambda x: isinstance(x, pd.core.series.Series))].index,:]["Aliases"]

k_id = k_id.loc[k_id.apply(lambda x: isinstance(x, str))].unique() # kinase genes (2235)

# TAIR10 GFF genes
gff = dt.fread("../data/TAIR10/Athaliana_167_gene.gff3", skip_to_line=4).to_pandas()
at_id = gff.loc[gff["C2"]=="gene"].\
    apply(lambda x: x["C8"].replace("ID=", "").split(";")[0].strip() if isinstance(x["C8"], str) else x, axis=1)

# TAIR10 Kinase genes; generate all possible pairs
tair10_kinases = np.intersect1d(k_id, at_id) # 2182 kinase genes
pairs = list(itertools.combinations(tair10_kinases, 2)) # 2379471 pairs

# Save the pairs to a file
pd.DataFrame(pairs, columns=["gene1", "gene2"]).to_csv(
    "../data/Kinase_genes/instances_tair10_kinases.txt", index=False, sep="\t")


### Prepare instance file of Araport11 Kinase genes

In [ ]:
# Kinase genes from NCBI
genes = pd.read_csv("../data/Kinase_genes/kinase-athaliana-ncbi.tsv", sep="\t")
k_id = genes.apply(lambda x: x["Aliases"].split(",")[0].strip() if isinstance(x["Aliases"], str) else x, axis=1)

print(k_id.apply(type).unique()) # type: ignore
# [<class 'str'> <class 'pandas.core.series.Series'>]
genes.iloc[k_id.loc[k_id.apply(lambda x: isinstance(x, pd.core.series.Series))].index,:]["Aliases"] # these Aliases are NaN values

k_id = k_id.loc[k_id.apply(lambda x: isinstance(x,str))].unique() # kinase genes (2235)

# Araport11 GFF genes
gff = dt.fread("../data/Araport11/Athaliana_447_Araport11.gene.gff3", skip_to_line=4).to_pandas()
at_id = gff.loc[gff["C2"]=="gene"].\
    apply(lambda x: x["C8"].replace("ID=", "").split(".")[0].strip() if isinstance(x["C8"], str) else x, axis=1)

# Araport11 Kinase genes; generate all possible pairs
araport11_kinases = np.intersect1d(k_id, at_id) # 2202 kinase genes
pairs = list(itertools.combinations(araport11_kinases, 2)) # 2423301 pairs

# Save the pairs to a file
pd.DataFrame(pairs, columns=["gene1", "gene2"]).to_csv(
    "../data/Kinase_genes/instances_araport11_kinases.txt", index=False, sep="\t")


### Prepare instance file for single genes

In [2]:
import gffutils

gff = '../data/TAIR10/Athaliana_167_TAIR10.gene.gff3'

# Create the database
db = gffutils.create_db(gff, dbfn='TAIR10.db', force=True, keep_order=True,
                        merge_strategy='merge', sort_attribute_values=True) 

# Extract the gene information
db = gffutils.FeatureDB('../data/TAIR10/TAIR10.db') # access the database
genes = []
for gene in db.features_of_type('gene'):
    genes.append(gene['Name'][0])

print(len(genes)) # 27416

# Write the gene IDs to a file
# with open('../data/instances_dataset_singles.txt', 'w') as f:
#     f.write('gene\n')
#     for gene in genes:
#         f.write('%s\n' % gene)

27416


### Prepare instance file no. 2

In [3]:
from itertools import combinations

# Generate all possible gene pairs without duplicates
gene_pairs = list(combinations(genes, 2))
len(gene_pairs) # 375804820

# Write the gene pairs to a file
# with open('../data/instances_dataset_pairs.txt', 'w') as f:
#     f.write('gene1\tgene2\n')
#     for gene_pair in gene_pairs:
#         f.write('%s\t%s\n' % gene_pair)

375804820